# Natura - Reconhecimento de Produto Nomeado

In [1]:
import unicodedata, sklearn, pandas

from tqdm import tqdm
from utils import match
from sklearn.feature_extraction.text import TfidfVectorizer

tqdm.pandas()
pandas.set_option('display.max_colwidth', None)

## Carregando Dados - Nomes Diferentes

In [2]:
df1 = pandas.read_csv('nomes_diferentes.csv', sep=',')
df1.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO


## Carregando Dados - SKUs Diferentes

In [3]:
df2 = pandas.read_csv('skus_diferentes.csv', sep=',')
df2.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM
0,2024-09,103342,AMOSTRA,ESSENCIAL
1,2024-09,103347,AMOSTRA,ESSENCIAL
2,2024-11,103377,AMOSTRA,TODODIA


## Unindo Datasets

In [4]:
df = pandas.concat([df1, df2])

def remove_accents(input_str):
    if isinstance(input_str, str):
        return ''.join(
            c for c in unicodedata.normalize('NFD', input_str) if unicodedata.category(c) != 'Mn'
        )
    return input_str

df['NAME'] = (
    df['DC_PRODUTO_ITEM']
    .apply(remove_accents)
    .str.lower()
    .str.replace(r'[^a-z0-9\s]', ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.upper()
)

df['NAME'] = df['NAME'].astype(str)

df.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML


In [5]:
df

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
3,2024-3,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
4,2024-05,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
...,...,...,...,...,...
2253,2023-01,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2254,2023-09,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2255,2023-10,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2256,2023-04,95047,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML


## Exemplo - Estudo de Caso

In [6]:
print(f'Records: {len(df)}')

df[df['DC_PRODUTO_ITEM'].isin([
    'DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML',
    'HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML'
])]

Records: 21918


,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
46,2024-06,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
47,2024-08,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
48,2024-05,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
49,2024-09,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
50,2024-10,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
51,2024-7,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
52,2024-11,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
58,2023-02,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML
59,2023-03,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML
60,2023-06,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML


## Dataset Base

In [7]:
df_base = df.groupby('NAME')['NAME'].count().reset_index(name='COUNT')
df_base['ID'] = range(0, len(df_base))
df_base = df_base[['ID', 'NAME']]

print(f'Records: {len(df_base)}')
df_base.head(3)

Records: 2191


,ID,NAME
0,0,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM ESSENCE 75 ML
1,1,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM SAGAZ 75 ML
2,2,AGUA DE BANHO EKOS PRIPRIOCA 200 ML


## Resolvendo Nomes

In [8]:
model = TfidfVectorizer().fit(df_base['NAME'])
model

TfidfVectorizer()

In [9]:
df_base['_ID'] = df_base['NAME'].progress_apply(lambda x : match(model=model, df=df_base, _str=x, threshold=1.0))
df_base

100%|██████████| 2191/2191 [1:00:23<00:00,  1.65s/it]  


,ID,NAME,_ID
0,0,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM ESSENCE 75 ML,0
1,1,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM SAGAZ 75 ML,1
2,2,AGUA DE BANHO EKOS PRIPRIOCA 200 ML,2
3,3,AGUA DE COLONIA MAMAE E BEBE FLOR DE LARANJEIRA 100 ML,3
4,4,AGUA DE COLONIA PAPAI E BEBE 100 ML,4
...,...,...,...
2186,2186,VELA PERFUMADA NATURA 774 ROSA CAPITIU 170 G,2186
2187,2187,VELA PERFUMADA NATURA 861 GUAIACO PATAQUEIRA 170 G,2187
2188,2188,VOVO DES COL MASC 100ML 2024,2188
2189,2189,VOVO DESOD COL FEM 100ML 2024,2189


In [17]:
dfm = df.merge(df_base, on='NAME', how='left')
dfm = dfm.drop('ID', axis=1).rename(columns={'_ID': 'ID'})

dfm.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME,ID
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984


In [96]:
df_sample = dfm.copy()
df_sample['ANOMES'] = pandas.to_datetime(df_sample['ANOMES'], format='%Y-%m')

df_sample = (
    df_sample
        .sort_values(by='ANOMES', ascending=False)
        .groupby(['ID', 'NAME'], as_index=False)
        .agg({
            'CD_PRODUTO_ITEM': lambda x: list(set(x)),
            'DC_PRODUTO_ITEM': lambda x: list(set(x))
        })
        .reset_index()
)

df_sample = df_sample[
    (df_sample['ID'] != -1) &
    (df_sample['CD_PRODUTO_ITEM'].apply(lambda x: len(x) > 1))
]

df_sample = df_sample[df_sample['NAME'] != 'AMOSTRA']
df_sample = df_sample.sample(n=50, random_state=42)
df_sample

,index,ID,NAME,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM
294,294,255,CONDICIONADOR MAMAE E BEBE 200 ML,"[92793, 92822]",[CONDICIONADOR MAMÃE E BEBÊ 200 ML]
159,159,143,BATOM MATTE INTRANSFERIVEL UNA,"[54394, 54355]",[BATOM MATTE INTRANSFERÍVEL UNA]
2120,2120,2106,SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORACAO LUMINA 150 ML,"[148449, 140754]","[SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORAÇÃO LUMINA 150 ML, SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORAÇÃO LUMINA - 150 ML]"
1122,1122,1086,KIT OLEO DESODORANTE CORPORAL SEVE AMENDOAS E CANELA 2 UNIDADES,"[138809, 151522, 134274]",[KIT ÓLEO DESODORANTE CORPORAL SÈVE AMÊNDOAS E CANELA - 2 UNIDADES]
1108,1108,1072,KIT DESODORANTE ANTITRANSPIRANTE ROLL ON TODODIA MANGA ROSA E AGUA DE COCO 2 UN DE 70 ML,"[154752, 155526]",[KIT DESODORANTE ANTITRANSPIRANTE ROLL-ON TODODIA MANGA ROSA E ÁGUA DE COCO - 2 UN. DE 70 ML]
1139,1139,1103,KIT SHAMPOO MAMAE E BEBE,"[544, 550]",[KIT SHAMPOO MAMÃE E BEBÊ]
1698,1698,1680,REFIL MASCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA 250 ML,"[117092, 148439]","[REFIL MÁSCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA - 250 ML, REFIL MÁSCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA 250 ML]"
1520,1520,1500,PRESENTE NATURA TODODIA FLOR DE LIS,"[128513, 126156]",[PRESENTE NATURA TODODIA FLOR DE LIS]
1504,1504,1484,PRESENTE NATURA KAIAK FEMININO,"[134856, 138768, 125610]",[PRESENTE NATURA KAIAK FEMININO]
1764,1764,1748,REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA 300 ML,"[148169, 112026]","[REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA - 300 ML, REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA 300 ML]"


In [97]:
df_most_recent = df[['NAME', 'ANOMES', 'CD_PRODUTO_ITEM']].copy()
df_most_recent['ANOMES'] = pandas.to_datetime(df_most_recent['ANOMES'], format='%Y-%m')

df_most_recent = (
    df_most_recent
        .sort_values(by='ANOMES', ascending=False)
        .groupby('NAME', as_index=False)
        .first()
)

df_most_recent

,NAME,ANOMES,CD_PRODUTO_ITEM
0,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM ESSENCE 75 ML,2024-10-01,134534
1,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM SAGAZ 75 ML,2024-06-01,129623
2,AGUA DE BANHO EKOS PRIPRIOCA 200 ML,2024-11-01,123049
3,AGUA DE COLONIA MAMAE E BEBE FLOR DE LARANJEIRA 100 ML,2024-11-01,92789
4,AGUA DE COLONIA PAPAI E BEBE 100 ML,2024-11-01,92824
...,...,...,...
2186,VELA PERFUMADA NATURA 774 ROSA CAPITIU 170 G,2024-11-01,151029
2187,VELA PERFUMADA NATURA 861 GUAIACO PATAQUEIRA 170 G,2024-10-01,151027
2188,VOVO DES COL MASC 100ML 2024,2024-07-01,133874
2189,VOVO DESOD COL FEM 100ML 2024,2024-08-01,133873


In [98]:
df_most_recent[df_most_recent['NAME'] == 'PRESENTE NATURA TODODIA ALGODAO']

,NAME,ANOMES,CD_PRODUTO_ITEM
1497,PRESENTE NATURA TODODIA ALGODAO,2024-02-01,135001


In [99]:
df_final = df_sample.merge(df_most_recent, on='NAME', how='left')

df_final = df_final.rename(columns={
    'NAME': 'nm_produto',
    'CD_PRODUTO_ITEM_y': 'cd_produto',
    'CD_PRODUTO_ITEM_x': 'todos_cds',
    'DC_PRODUTO_ITEM': 'todos_nms',
    'ANOMES': 'dt_ultima_atualizacao'
})

df_final = df_final.drop('ID', axis=1).reset_index()
df_final = df_final[['cd_produto', 'nm_produto', 'todos_cds', 'todos_nms', 'dt_ultima_atualizacao']]

df_final

,cd_produto,nm_produto,todos_cds,todos_nms,dt_ultima_atualizacao
0,92793,CONDICIONADOR MAMAE E BEBE 200 ML,"[92793, 92822]",[CONDICIONADOR MAMÃE E BEBÊ 200 ML],2024-11-01
1,54355,BATOM MATTE INTRANSFERIVEL UNA,"[54394, 54355]",[BATOM MATTE INTRANSFERÍVEL UNA],2024-11-01
2,148449,SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORACAO LUMINA 150 ML,"[148449, 140754]","[SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORAÇÃO LUMINA 150 ML, SPRAY ANTIDESBOTAMENTO E BRILHO PARA CABELOS OPACOS OU COM COLORAÇÃO LUMINA - 150 ML]",2024-11-01
3,138809,KIT OLEO DESODORANTE CORPORAL SEVE AMENDOAS E CANELA 2 UNIDADES,"[138809, 151522, 134274]",[KIT ÓLEO DESODORANTE CORPORAL SÈVE AMÊNDOAS E CANELA - 2 UNIDADES],2024-02-01
4,155526,KIT DESODORANTE ANTITRANSPIRANTE ROLL ON TODODIA MANGA ROSA E AGUA DE COCO 2 UN DE 70 ML,"[154752, 155526]",[KIT DESODORANTE ANTITRANSPIRANTE ROLL-ON TODODIA MANGA ROSA E ÁGUA DE COCO - 2 UN. DE 70 ML],2023-11-01
5,544,KIT SHAMPOO MAMAE E BEBE,"[544, 550]",[KIT SHAMPOO MAMÃE E BEBÊ],2023-08-01
6,117092,REFIL MASCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA 250 ML,"[117092, 148439]","[REFIL MÁSCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA - 250 ML, REFIL MÁSCARA RECONSTRUTORA PARA CABELOS CRESPOS LUMINA 250 ML]",2024-11-01
7,126156,PRESENTE NATURA TODODIA FLOR DE LIS,"[128513, 126156]",[PRESENTE NATURA TODODIA FLOR DE LIS],2023-06-01
8,134856,PRESENTE NATURA KAIAK FEMININO,"[134856, 138768, 125610]",[PRESENTE NATURA KAIAK FEMININO],2024-01-01
9,148169,REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA 300 ML,"[148169, 112026]","[REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA - 300 ML, REFIL SHAMPOO HIDRATANTE PARA CABELOS CRESPOS LUMINA 300 ML]",2024-11-01


In [100]:
df_final.to_csv('~/Downloads/amostra_skus_consolidados.csv', sep=';', index=False)